<a href="https://colab.research.google.com/github/taepd/sba-chatbot/blob/youngdon/%ED%98%91%EC%97%85%ED%95%84%ED%84%B0%EB%A7%81_%EC%97%B0%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd


- 데이터셋 로드






In [2]:
df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/ratings.dat", sep="::", encoding='utf-8')


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [4]:
df.head()

,1,1193,5,978300760
0,1,661,3,978302109
1,1,914,3,978301968
2,1,3408,4,978300275
3,1,2355,5,978824291
4,1,1197,3,978302268


- 컬럼명 추가


In [3]:
df.columns = ["user_id", "item_id", "rating", "timestamp"]
df.head()

,user_id,item_id,rating,timestamp
0,1,661,3,978302109
1,1,914,3,978301968
2,1,3408,4,978300275
3,1,2355,5,978824291
4,1,1197,3,978302268


- shape 확인

In [6]:
df.shape

(1000208, 4)

- groupby로 rating 별 user_id를 count

In [7]:
df.groupby(["rating"])[["user_id"]].count()

,user_id
rating,
1,56174
2,107557
3,261197
4,348971
5,226309


- groupby로 item_id 별 user_id를 count

In [8]:
df.groupby(["item_id"])[["user_id"]].count().head()

,user_id
item_id,
1,2077
2,701
3,478
4,170
5,296


- 중복 제거한 user_id 리스트 생성
- user_id를 중복을 제거해서 count

In [4]:
unique_user_list = df.user_id.unique()
n_users = unique_user_list.shape[0]
print(unique_user_list)
n_users

[   1    2    3 ... 6038 6039 6040]


6040

- item_id를 중복을 제거
   - item_id는 dense하지 않으므로 가장 큰 item_id값에 맞춰서 크기 설정

In [5]:
unique_item_list = df.item_id.unique()
n_items = np.max(unique_item_list)
print(unique_item_list)
print(np.sort(unique_item_list))
n_items

[ 661  914 3408 ... 2845 3607 2909]
[   1    2    3 ... 3950 3951 3952]


3952

- shape(n_users, n_items) ratings 영행렬의 평가행렬 생성

In [6]:
ratings = np.zeros((n_users, n_items))
ratings.shape

(6040, 3952)

- ratings를 sparse matrix 형태로 구성
- df.itertuples(): dataframe의 각 행을 Pandas 객체로 리턴
  - 컬럼명으로도 접근 가능하고, 인덱스로도 접근 가능
  - 단, 인덱스는 0부터 시작하므로 -1씩 해준다.


In [7]:
for row in df.itertuples():
    ratings[row[1]-1, row[2]-1] = row[3]

ratings

array([[5., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [3., 0., 0., ..., 0., 0., 0.]])

- scikit-learn 라이브러리를 활용해 훈련 데이터와 테스트 데이터를 분리

In [23]:
from sklearn.model_selection import train_test_split
# random_state: seed 부여 랜덤하게 만들되, 해당 시드일 경우는 항상 같음
ratings_train, ratings_test = train_test_split(ratings, test_size=0.33, random_state=37)

In [25]:
print(ratings_train)
ratings_train.shape

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


(4046, 3952)

- 사용자 간 유사도 행렬 생성
- 코사인 유사도를 사용
  - cosine_distances는 유사할수록 값이 0에 가까움
  - cosine_similarity는 유사할수록 값이 1에 가까움
    - cosine_similarty = 1- cosine_distances

In [15]:
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

user_similarity = cosine_similarity(ratings_train)
user_similarity

array([[1.        , 0.16332531, 0.15564215, ..., 0.1268875 , 0.16999306,
        0.07643071],
       [0.16332531, 1.        , 0.02907329, ..., 0.01693003, 0.        ,
        0.04150271],
       [0.15564215, 0.02907329, 1.        , ..., 0.20570353, 0.18383079,
        0.09254779],
       ...,
       [0.1268875 , 0.01693003, 0.20570353, ..., 1.        , 0.11497834,
        0.09120296],
       [0.16999306, 0.        , 0.18383079, ..., 0.11497834, 1.        ,
        0.04665282],
       [0.07643071, 0.04150271, 0.09254779, ..., 0.09120296, 0.04665282,
        1.        ]])

## 평가 예측

- 트레이닝 행렬과 사용자간 유사도 행렬간 내적연산(dot())을 함

In [16]:
print('사용자간 유사도 행렬:', user_similarity.shape)
print('트레이닝 행렬:', ratings_train.shape)
user_similarity_dot_ratings_train = user_similarity.dot(ratings_train)

사용자간 유사도 행렬: (4046, 4046)
트레이닝 행렬: (4046, 3952)


- 사용자간 유사도 행렬의 열(axis=1)로 즉, 아이템을 기준으로 모두 더해준 array를 생성

In [17]:
sum_by_item_id_array = np.array([np.abs(user_similarity).sum(axis=1)])
print(sum_by_item_id_array.shape)
sum_by_item_id_array

(1, 4046)


array([[585.8834592 , 424.7353442 , 444.8792355 , ..., 551.71433632,
        504.70031636, 236.6528167 ]])

- 예측값을 user_similarity_dot_ratings_train에 sum_by_item_id_array를 전치하여 나누어 계산
- 즉, 유사도에 따라 가중치를 부여하여 item_id에 따른 유사도 합으로 나누어주어 예측값 생성




In [18]:
user_pred = user_similarity_dot_ratings_train / sum_by_item_id_array.T
print(user_pred.shape)
user_pred

(4046, 3952)


array([[1.68895336, 0.50379711, 0.29182767, ..., 0.03816307, 0.02028697,
        0.25616819],
       [1.79553346, 0.50003617, 0.29031143, ..., 0.04325957, 0.02027759,
        0.26266574],
       [1.65079625, 0.43592557, 0.29733319, ..., 0.04376734, 0.02937503,
        0.28239199],
       ...,
       [1.68775123, 0.40260552, 0.27259654, ..., 0.04694083, 0.02995773,
        0.31211325],
       [1.85444198, 0.56200933, 0.38157914, ..., 0.04985682, 0.02462044,
        0.29571103],
       [1.48862241, 0.38430593, 0.34920859, ..., 0.05290756, 0.04211202,
        0.36622238]])

 ## 모델의 성능 평가

- RMSE(mean_squared_error)로 성능 평가(오차 계산)

- scikit-learn을 활용해 MSE 함수 구현 
- 예측 행렬과 트레이닝 행렬의 요소 중 0이 아닌 값들을 뽑아서 mean_squared_error()에 인자로 넣어줌
  - numpy nonzero()메소드를 이용해 트레이닝 행렬의 nonzero 요소값을 구하고, 예측 행렬과 트레이닝 행렬에서 이를 만족하는 요소들의 array를 만듬

In [19]:
print(user_pred[ratings_train.nonzero()].shape) # array이기 때문에 flatton()하지 않아도 됨
user_pred[ratings_train.nonzero()].shape

(675264,)


(675264,)

In [34]:
from sklearn.metrics import mean_squared_error

def get_mse(pred, actual):
  pred = pred[actual.nonzero()]
  actual = actual[actual.nonzero()]
  return mean_squared_error(pred, actual)

- RMSE 측정
- 트레이닝 오차

In [21]:
np.sqrt(get_mse(user_pred, ratings_train))

3.0354269774044904

- 테스트 오차

In [22]:
np.sqrt(get_mse(user_pred, ratings_test))

3.123936391794805

## 모든 트레이닝 세트 샘플을 대상으로 예측을 한 결과 오차가 너무 크게 나옴. 따라서 타겟과 가장 유사한 n개의 샘플을 대상을 선택하여 예측 시도

- NearestNeighbors: knn과 유사하나 비지도 방식
  - 코사인 유사도로 측정

In [10]:
from sklearn.neighbors import NearestNeighbors

k = 100

neigh = NearestNeighbors(n_neighbors=k, metric="cosine")

neigh.fit(ratings_train)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=100, p=2,
                 radius=1.0)

In [24]:
top_k_distances, top_k_users = neigh.kneighbors(ratings_train, return_distance=True)

print(top_k_distances)
print(top_k_users)
print(top_k_distances.shape)
print(top_k_users.shape)

[[0.         0.54744567 0.56686352 ... 0.69492561 0.6950103  0.69507526]
 [0.         0.58697903 0.60829454 ... 0.75019329 0.75022381 0.7505434 ]
 [0.         0.5577343  0.57337391 ... 0.73184394 0.73188588 0.73192529]
 ...
 [0.         0.52406512 0.58125917 ... 0.70387883 0.70417809 0.70430272]
 [0.         0.54335305 0.56555768 ... 0.69613884 0.69619714 0.69747499]
 [0.         0.61894678 0.64071109 ... 0.82329665 0.82355578 0.82411754]]
[[   0 3437 2078 ... 1748 2753 3825]
 [   1 4022 3976 ... 4035  635 3729]
 [   2 3301 2989 ... 2178 1634 1999]
 ...
 [4043  363  267 ... 1159 3413 1370]
 [4044 1987 3448 ... 1311  422  539]
 [4045 3788 3414 ... 3302  375  753]]
(4046, 100)
(4046, 100)


- 선택된 n명의 사용자들의 평가 가중치 합을 사용한 예측 및 모델의 성능 측정

- shape 체크 부분(흐름과 상관없음)

In [25]:
print(np.array([np.abs(top_k_distances[1]).sum()]))
print(np.array([np.abs(top_k_distances[1].T).sum(axis=0)]).shape)
np.array([np.abs(top_k_distances[1].T).sum(axis=0)]).T.shape
print(top_k_distances[1].T.shape)
print(ratings_train[top_k_users[1]].shape)
# top_k_distances[i].T.dot(ratings_train[top_k_users][i]).shape

[71.1230402]
(1,)
(100,)
(100, 3952)


- 한 줄씩 계산하므로 벡터이기 때문에 굳이 전치하지 않아도 결과는 같다

In [26]:
# user_pred_k_1 = top_k_distances[1].dot(ratings_train[top_k_users][1]) / \
#                       np.array([np.abs(top_k_distances[1]).sum(axis=0)])
# print(user_pred_k_1)
# user_pred_k_2 = top_k_distances[1].T.dot(ratings_train[top_k_users][1]) / \
#                       np.array([np.abs(top_k_distances[1].T).sum(axis=0)]).T
# print(user_pred_k_2)
# (user_pred_k_1 == user_pred_k_2).min()

In [27]:
user_pred_k = np.zeros(ratings_train.shape)
user_pred_k.shape

(4046, 3952)

In [ ]:

for i in range(ratings_train.shape[0]):  # 4046: 테스트 샘플 수
  if i%50 == 0: # 계산이 오래 걸리므로 카운터 체크
    print("cnt = ", i)
  # top_k_distances[i].T.shape = (5, )
  # ratings_train[top_k_users[i]] = (5, 3952)
  # user_pred_k[i, :] = top_k_distances[i].T.dot(ratings_train[top_k_users][i]) / \
  #                     np.array([np.abs(top_k_distances[i].T).sum(axis=0)]).T                    
  user_pred_k_1 = top_k_distances[1].dot(ratings_train[top_k_users][1]) / \
                      np.array([np.abs(top_k_distances[1]).sum(axis=0)])                                        

cnt =  0


- RMSE 측정
- 트레이닝 오차

In [ ]:
np.sqrt(get_mse(user_pred_k, ratings_train))

- 테스트 오차

In [ ]:
np.sqrt(get_mse(user_pred_k, ratings_test))

## 아이템 기반 협업 필터링 사용 추천 엔진

- 영화의 수를 k로 사용해 영화 간 유사도 행렬 계산

In [17]:
k = ratings_train.shape[1]
print('k: ', k)
neigh = NearestNeighbors(n_neighbors=k, metric="cosine")

k:  3952


In [27]:
print(ratings_train.T.shape)
neigh.fit(ratings_train.T)

(3952, 4046)


NearestNeighbors(algorithm='auto', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=3952, p=2,
                 radius=1.0)

In [30]:
item_distances, _ = neigh.kneighbors(ratings_train.T, return_distance=True)

print(item_distances.shape)
print(item_distances[1, 0])
item_distances

(3952, 3952)
0.0


array([[8.88178420e-16, 3.68442811e-01, 3.86433672e-01, ...,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
       [0.00000000e+00, 4.40673888e-01, 5.08666076e-01, ...,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
       [0.00000000e+00, 4.51909601e-01, 6.30625590e-01, ...,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
       ...,
       [0.00000000e+00, 6.68115700e-01, 6.98463564e-01, ...,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
       [0.00000000e+00, 6.45419721e-01, 6.75422482e-01, ...,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
       [1.11022302e-16, 5.50665177e-01, 5.79996952e-01, ...,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00]])

In [31]:
item_pred = ratings_train.dot(item_distances) / np.array([np.abs(item_distances).sum(axis=1)])

In [32]:
item_pred.shape

(4046, 3952)

In [35]:
np.sqrt(get_mse(item_pred, ratings_train))

3.4381175344988635

In [36]:
np.sqrt(get_mse(item_pred, ratings_test))

3.6155887611395037